In [1]:
from spectrm_challenge_ryan.helpers import SpectrmLoader
from spectrm_challenge_ryan.models import WordFrequency

In [2]:
loader = SpectrmLoader()
data = loader.load()

In [69]:
wf_corpus = WordFrequency()
wf_corpus.get_unique_non_stop_words(data['train_dialogs'])

In [71]:
labels = data['train_dialogs'].index.unique()

In [75]:
print len(labels)
print len(wf_corpus.unique_non_stop_words)

13731
26532


In [ ]:
input_mat = {}
target_mat = {}
wf_input = WordFrequency()
wf_target = WordFrequency()
for label in labels:
    wf_input.get_unique_non_stop_words(data['train_dialogs'].ix[label])
    wf_target.get_unique_non_stop_words(data['train_missing'].ix[label].text)
    input_mat[label] = wf_corpus.unique_non_stop_words.isin(wf_input.unique_non_stop_words)
    target_mat[label] = wf_corpus.unique_non_stop_words.isin(wf_target.unique_non_stop_words)